<a href="https://colab.research.google.com/github/w86w86/econ8310-assignment1/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1 - Forecast the # of taxi trips requested during each hour in a week

##Execution

In [43]:
## Import statements
import pandas as pd
import numpy as np
from prophet import Prophet

## Import the dataset of #of taxi trips each hour in New York City,
file = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv"
trip = pd.read_csv(file)

## Keep only the dates and the y value
trip = trip[['Timestamp','trips']]
trip.Timestamp = pd.to_datetime(trip.Timestamp)

## Prophet format using column ds and y
trip=trip.rename(columns={'Timestamp':'ds','trips':'y'})

## Prophet instance and fit to data / modelFit
model = Prophet()
modelFit = model.fit(trip)

## Create timeline 31 days and forecast
future = modelFit.make_future_dataframe(periods=744, freq='h', include_history=False)
forecast = modelFit.predict(future)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqlols3i4/dmhylt18.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqlols3i4/tww4m8pk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10618', 'data', 'file=/tmp/tmpqlols3i4/dmhylt18.json', 'init=/tmp/tmpqlols3i4/tww4m8pk.json', 'output', 'file=/tmp/tmpqlols3i4/prophet_modeluscxxst4/prophet_model-20250917072540.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:25:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:25:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [55]:
pred = forecast.yhat.values

## Replace the negative value with min of trip in an hour from our data
pred = [max(min(trip.y), val) for val in pred]

print(pred)

[np.float64(7618.173261325992), np.float64(5137.799878266395), np.float64(2657.221235155728), np.float64(862.5983133588088), np.float64(622.0615299630026), np.float64(2323.1878633341084), np.float64(5432.466880498393), np.float64(8730.090174217145), np.float64(11086.572933106278), np.float64(12153.355469466596), np.float64(12411.553810279162), np.float64(12598.368131877553), np.float64(13067.043165176627), np.float64(13636.835141803982), np.float64(13979.425007708007), np.float64(14078.101012276053), np.float64(14273.787061355999), np.float64(14869.839151168431), np.float64(15727.331478061968), np.float64(16284.124028255581), np.float64(15989.554577890434), np.float64(14724.26516661284), np.float64(12803.310182515459), np.float64(10602.766349385412), np.float64(8241.461710554822), np.float64(5678.28943734351), np.float64(3119.164381804826), np.float64(1250.944139435669), np.float64(942.3756307830063), np.float64(2581.566879232396), np.float64(5635.4487897118115), np.float64(8884.561474

#Tests From Dr White

###imports and Run Function Check

In [ ]:
#!pip install pygam

In [57]:
import unittest
import patsy as pt
from pygam import LinearGAM, f, s, l
import numpy as np
import pandas as pd
import statsmodels as statsmodels
import pygam
from statsmodels.tsa.holtwinters import ExponentialSmoothing as ES
import prophet
import statsmodels.tsa.holtwinters as hw
from statsmodels.tsa.statespace.varmax import VARMAXResultsWrapper
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper

# Import your code from parent directory
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

#from assignment1 import model, modelFit, pred

In [46]:
#bool(modelFit.history) ??

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [61]:
# Run the checks
def checkNumbers(series):
  for i in series:
    if not isinstance(i, (float, int)):
      return False
  return True


def test_valid_model():
    return ( isinstance(model, prophet.forecaster.Prophet), "Make sure that you are using a model\ncovered in class in the variable 'model'.")

def test_fitted_model():
    if isinstance(modelFit, VARResultsWrapper) or isinstance(modelFit, VARMAXResultsWrapper) or isinstance(modelFit, hw.results.HoltWintersResultsWrapper):
        return (True)
    elif hasattr(modelFit, '_is_fitted'):
        if modelFit._is_fitted:
            return (True)
    elif hasattr(modelFit, 'history'):
        return (isinstance(modelFit.history, pd.DataFrame))
    else:
        print("Make sure that you store your fitted model in the variable 'modelFit'.")
        return (False)

def test_valid_pred():
  return (len(np.squeeze(pred))==744 and checkNumbers(np.squeeze(pred))), "Make sure your prediction consists of integers\nor floating point numbers, and is a list or array of 744\nfuture predictions!"

def test_accuracy_level_1():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<220)

def test_accuracy_level_2():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<185)

def test_accuracy_level_3():
  dataTest = pd.read_csv(currentdir + "/testData.csv")['trips']
  rmse = sum([(np.squeeze(pred)[i]-dataTest[i])**2 for i in range(len(np.squeeze(pred)))])
  rmse = np.sqrt(rmse)*1/744

  return (rmse<171)


#plt = model.plot(forecast)

###run all tests if TRUE

In [62]:
test_valid_model()

(True,
 "Make sure that you are using a model\ncovered in class in the variable 'model'.")

In [63]:
test_fitted_model()

True

In [64]:
test_valid_pred()

(True,
 'Make sure your prediction consists of integers\nor floating point numbers, and is a list or array of 744\nfuture predictions!')

In [65]:
test_accuracy_level_1()

np.True_

In [66]:
test_accuracy_level_2()

np.True_

In [67]:
test_accuracy_level_3()

np.True_